In [4]:
!pip install shap

INFO: pip is looking at multiple versions of pandas to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/549.3 kB ? eta -:--:--
   -------------------------------------- - 524.3/549.3 kB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 549.3/549.3 kB 3.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.6 MB ? eta -:--:--
    --------------------------------------- 0.3/15.6 MB ? eta -:--:--
   --- ------------------------------------ 1.3/15.6 MB 3.5 MB/s eta 0:00:05
   ----- ---------------------------------- 2.1/15.6 MB 3.4 MB/s eta 0:00:05
   -------- ------------------------------- 3.1/15.6 MB 3.8 MB/s eta 0:00:04
   ---------- ----------------------------- 3.9/15.6 MB 4.2 MB/s eta 0:00:03
   -------------- ------------------------- 5.5/15.6 MB 4.4 MB/s eta 0:00:03
   ------------------ --------------------- 7.1/15.6 MB 4.8 MB/s eta 0:00:02
   ---------------------

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
accelerate 1.12.0 requires torch>=2.0.0, which is not installed.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.0.2 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.0.2 which is incompatible.
streamlit 1.37.1 requires pandas<3,>=1.3.0, but you have pandas 3.0.1 which is incompatible.
streamlit 1.37.1 requires protobuf<6,>=3.20, but you have protobuf 6.33.5 which is incompatible.


In [2]:
!pip install optuna

In [6]:
# =========================================
# Phase 5 — Deep Learning (LSTM & TCN) + Optuna + SHAP
# =========================================

import os
import json 
import csv  # اضافه شده برای دور زدن باگ پانداز
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from sklearn.metrics import mean_squared_error 
import optuna
import shap

# -------------------------
# A) Config
# -------------------------
PHASE2_DIR = "artifacts/phase2"
OUT_DIR = "artifacts/phase5"
os.makedirs(OUT_DIR, exist_ok=True)

TRAIN_F = os.path.join(PHASE2_DIR, "train_features.csv")
VAL_F   = os.path.join(PHASE2_DIR, "val_features.csv")
TEST_F  = os.path.join(PHASE2_DIR, "test_features.csv")
FEAT_F  = os.path.join(PHASE2_DIR, "feature_cols.json")

LOOK_BACK = 14   
HORIZONS = 7     # افق پیش‌بینی 7 روزه
SAMPLE_N = 50    

# -------------------------
# B) Load Data & Store-Aware Sliding Window
# -------------------------
train_df = pd.read_csv(TRAIN_F)
val_df   = pd.read_csv(VAL_F)
test_df  = pd.read_csv(TEST_F)

with open(FEAT_F, "r", encoding="utf-8") as f:
    feature_cols = json.load(f)["feature_cols"]

# اطمینان از حذف Store از ویژگی‌های آموزشی
if "Store" in feature_cols:
    feature_cols.remove("Store")

TARGET = "Sales"

def create_sequences_store_aware(df, look_back):
    X, y = [], []
    # گروه‌بندی بر اساس فروشگاه تا دیتای دو فروشگاه با هم قاطی نشود
    for store_id, group in df.groupby("Store"):
        group = group.sort_values("Date")
        X_data = group[feature_cols].to_numpy(dtype=np.float32)
        y_data = group[TARGET].to_numpy(dtype=np.float32)
        
        for i in range(len(X_data) - look_back):
            X.append(X_data[i : i + look_back])
            y.append(y_data[i + look_back])
            
    return np.array(X), np.array(y)

print("Building Sequences (This might take a moment)...")
X_train, y_train = create_sequences_store_aware(train_df, LOOK_BACK)
X_val, y_val     = create_sequences_store_aware(val_df, LOOK_BACK)
X_test, y_test   = create_sequences_store_aware(test_df, LOOK_BACK)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")

# -------------------------
# C & D) Build Models (Dynamic Architecture)
# -------------------------
def build_lstm_model(input_shape, units=128, dropout_rate=0.2):
    model = Sequential([
        LSTM(units, activation='relu', input_shape=input_shape),
        Dropout(dropout_rate),
        Dense(64, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def build_tcn_model(input_shape):
    model = Sequential([
        Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# -------------------------
# E) Hyperparameter Optimization (Optuna)
# -------------------------
def objective(trial):
    lstm_units = trial.suggest_int('lstm_units', 32, 128)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.4) 
    
    model = build_lstm_model((LOOK_BACK, len(feature_cols)), units=lstm_units, dropout_rate=dropout_rate)
    
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
    
    model.fit(X_train, y_train, epochs=5, batch_size=256, validation_data=(X_val, y_val), 
              callbacks=[early_stop], verbose=0)
    
    val_loss = model.evaluate(X_val, y_val, verbose=0)
    return val_loss

print("\nStarting Optuna optimization for LSTM...")
RUN_OPTUNA = False # برای صرفه‌جویی در زمان روی False تنظیم شده

if RUN_OPTUNA:
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=5)
    best_params = study.best_trial.params
else:
    best_params = {'lstm_units': 64, 'dropout_rate': 0.2}

print("Best LSTM params:", best_params)

# Train Final Models
print("\nTraining final LSTM model...")
lstm_model = build_lstm_model((LOOK_BACK, len(feature_cols)), 
                              units=best_params['lstm_units'], 
                              dropout_rate=best_params['dropout_rate'])
lstm_model.fit(X_train, y_train, epochs=10, batch_size=256, validation_data=(X_val, y_val), verbose=1)

print("\nTraining final TCN model...")
tcn_model = build_tcn_model((LOOK_BACK, len(feature_cols)))
tcn_model.fit(X_train, y_train, epochs=10, batch_size=256, validation_data=(X_val, y_val), verbose=1)

# -------------------------
# F & H) Forecasting & Metrics
# -------------------------
print("\nEvaluating on Test Set...")

lstm_preds_log = lstm_model.predict(X_test, verbose=0).flatten()
tcn_preds_log  = tcn_model.predict(X_test, verbose=0).flatten()

y_test_real = np.expm1(y_test)
lstm_preds_real = np.expm1(lstm_preds_log)
tcn_preds_real = np.expm1(tcn_preds_log)

def calculate_metrics(y_true, y_pred, model_name):
    rmse_val = np.sqrt(mean_squared_error(y_true, y_pred))
    
    mask = y_true > 0
    if np.any(mask):
        rmspe_val = np.sqrt(np.mean(((y_true[mask] - y_pred[mask]) / y_true[mask]) ** 2))
    else:
        rmspe_val = 0.0
        
    print(f"[{model_name}] RMSE: {rmse_val:.2f} Euros | RMSPE: {rmspe_val:.4f}")
    return rmse_val, rmspe_val

lstm_rmse, lstm_rmspe = calculate_metrics(y_test_real, lstm_preds_real, "LSTM")
tcn_rmse, tcn_rmspe   = calculate_metrics(y_test_real, tcn_preds_real, "TCN")

metrics_df = pd.DataFrame({
    "Model": ["LSTM", "TCN"],
    "RMSE_Euros": [lstm_rmse, tcn_rmse],
    "RMSPE": [lstm_rmspe, tcn_rmspe]
})

## --- FIX: جایگزینی pandas.to_csv با ماژول استاندارد csv برای دور زدن باگ ImportError --- ##
csv_path = os.path.join(OUT_DIR, "metrics_deep_learning.csv")
with open(csv_path, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(metrics_df.columns)
    for row in metrics_df.values:
        writer.writerow(row)

# رسم نمودار مقایسه‌ای برای 100 نمونه آخر
plot_samples = min(100, len(y_test_real))
plt.figure(figsize=(12, 5))
plt.plot(y_test_real[-plot_samples:], label="Actual Sales", color='black', linestyle='dashed', linewidth=2)
plt.plot(lstm_preds_real[-plot_samples:], label="LSTM Predictions", alpha=0.8)
plt.plot(tcn_preds_real[-plot_samples:], label="TCN Predictions", alpha=0.8)
plt.legend()
plt.title(f"Deep Learning Forecast vs Actuals (Last {plot_samples} Test Samples)")
plt.ylabel("Sales (Euros)")
plt.xlabel("Time Steps")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "dl_predictions_comparison.png"))
plt.close()

# -------------------------
# G) SHAP Integration
# -------------------------
print("\nGenerating SHAP values (Background context computing...)")
def shap_analysis(model, X_sample, model_name):
    X_sample_2d = X_sample.reshape((X_sample.shape[0], -1))
    
    def predict_wrapper(x_2d):
        x_3d = x_2d.reshape((x_2d.shape[0], LOOK_BACK, len(feature_cols)))
        return model.predict(x_3d, verbose=0).flatten() 
    
    background = shap.kmeans(X_sample_2d, 10)
    explainer = shap.KernelExplainer(predict_wrapper, background)
    
    shap_values = explainer.shap_values(X_sample_2d)
    
    plt.figure(figsize=(10,6))
    shap.summary_plot(shap_values, features=X_sample_2d, show=False)
    plt.title(f"SHAP Feature Importance ({model_name.upper()})")
    plt.savefig(os.path.join(OUT_DIR, f"shap_analysis_{model_name}.png"), bbox_inches='tight')
    plt.close()

shap_analysis(lstm_model, X_test[:20], "lstm")
shap_analysis(tcn_model, X_test[:20], "tcn")

summary_lines = [
    "Phase 5 outputs saved in artifacts/phase5/",
    f"LSTM Params -> Units: {best_params['lstm_units']}, Dropout: {best_params['dropout_rate']:.2f}",
    f"LSTM -> RMSE: {lstm_rmse:.2f} | RMSPE: {lstm_rmspe:.4f}",
    f"TCN  -> RMSE: {tcn_rmse:.2f} | RMSPE: {tcn_rmspe:.4f}",
    "SHAP plots and Prediction comparison plots generated successfully."
]

with open(os.path.join(OUT_DIR, "SUMMARY.txt"), "w", encoding="utf-8") as f:
    f.write("\n".join(summary_lines))

print("\n" + "="*50)
print("\n".join(summary_lines))
print("="*50)
print("Execution Completed Successfully! Project 100% Finished! 🚀")

Building Sequences (This might take a moment)...
X_train shape: (907939, 14, 59), y_train shape: (907939,)

Starting Optuna optimization for LSTM...
Best LSTM params: {'lstm_units': 64, 'dropout_rate': 0.2}

Training final LSTM model...


c:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
3547/3547 ━━━━━━━━━━━━━━━━━━━━ 31s 8ms/step - loss: 1.6344 - val_loss: 2.6919
Epoch 2/10
3547/3547 ━━━━━━━━━━━━━━━━━━━━ 30s 8ms/step - loss: 0.4128 - val_loss: 2.3341
Epoch 3/10
3547/3547 ━━━━━━━━━━━━━━━━━━━━ 52s 15ms/step - loss: 0.3413 - val_loss: 2.1829
Epoch 4/10
3547/3547 ━━━━━━━━━━━━━━━━━━━━ 61s 17ms/step - loss: 0.3084 - val_loss: 2.3877
Epoch 5/10
3547/3547 ━━━━━━━━━━━━━━━━━━━━ 48s 13ms/step - loss: 0.2886 - val_loss: 2.8398
Epoch 6/10
3547/3547 ━━━━━━━━━━━━━━━━━━━━ 47s 13ms/step - loss: 0.2752 - val_loss: 1.5392
Epoch 7/10
3547/3547 ━━━━━━━━━━━━━━━━━━━━ 46s 13ms/step - loss: 0.2652 - val_loss: 1.2952
Epoch 8/10
3547/3547 ━━━━━━━━━━━━━━━━━━━━ 46s 13ms/step - loss: 0.2513 - val_loss: 1.1611
Epoch 9/10
3547/3547 ━━━━━━━━━━━━━━━━━━━━ 49s 14ms/step - loss: 0.2502 - val_loss: 1.2560
Epoch 10/10
3547/3547 ━━━━━━━━━━━━━━━━━━━━ 48s 14ms/step - loss: 0.2438 - val_loss: 1.2652

Training final TCN model...


c:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
3547/3547 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - loss: 1.0917 - val_loss: 1.2483
Epoch 2/10
3547/3547 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - loss: 0.4394 - val_loss: 1.4830
Epoch 3/10
3547/3547 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - loss: 0.3984 - val_loss: 1.1917
Epoch 4/10
3547/3547 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - loss: 0.3779 - val_loss: 1.6381
Epoch 5/10
3547/3547 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - loss: 0.3650 - val_loss: 1.3591
Epoch 6/10
3547/3547 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - loss: 0.3534 - val_loss: 1.5059
Epoch 7/10
3547/3547 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - loss: 0.3436 - val_loss: 1.6966
Epoch 8/10
3547/3547 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - loss: 0.3373 - val_loss: 1.6671
Epoch 9/10
3547/3547 ━━━━━━━━━━━━━━━━━━━━ 16s 5ms/step - loss: 0.3306 - val_loss: 2.3264
Epoch 10/10
3547/3547 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - loss: 0.3250 - val_loss: 1.8963

Evaluating on Test Set...
[LSTM] RMSE: 3631.13 Euros | RMSPE: 0.4835
[TCN] RMSE: 2489.15 Euros | RMSPE: 0.37

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


  0%|          | 0/20 [00:00<?, ?it/s]

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


  0%|          | 0/20 [00:00<?, ?it/s]


Phase 5 outputs saved in artifacts/phase5/
LSTM Params -> Units: 64, Dropout: 0.20
LSTM -> RMSE: 3631.13 | RMSPE: 0.4835
TCN  -> RMSE: 2489.15 | RMSPE: 0.3768
SHAP plots and Prediction comparison plots generated successfully.
Execution Completed Successfully! Project 100% Finished! 🚀
